<a href="https://colab.research.google.com/github/YuryChet/education/blob/main/ML_DL_lesson_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 2

В этом задании:

1. Сделаем регрессию над данными через scikit-learn: сначала через регресию, потом через бустинг.
2. Сравним результаты с константным предсказанием.
3. Сделаем нейронную сеть на полносвязных слоях, обучим над теми же данными - и сравним с лин. регрессией и бустингом.



In [14]:
import pandas as pd
import numpy as np
import random

# Для воспроизводимости
seed = 0
np.random.seed(seed)
random.seed(seed)

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset/insurance.csv")

In [17]:
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [18]:
df.shape

(1338, 7)

### Задание №1:
Cделайте train/test split на данных в пропорции 0.2/0.8, залейте в лмс код, который в `df_train`, `df_test`
сохранит датафрейм с тренировочными и тестовыми данными соответственно.

P.S Использовать train_test_split из scikit-learn запрещено - разбивайте вручную через индексы.

In [19]:
# Разбиение датасета

# функцией randint берем рандомные числа от 0 до размера датафрейма, в количестве целог числа от 0.2*df.shape[0]
# короче тут получаются рандомные значения в количесте 20% от датафрейма
idx_test = np.random.choice(df.shape[0], size=int(0.2*df.shape[0]), replace=False)

#он берет индексы из списка idx_test
df_test = df.iloc[idx_test]

# сбрасывает индексы из списка у df и оставляет все остальное
df_train = df.drop(index=idx_test)




### Задание №2:
Сделайте `OHE` на колонки sex, region, smoker.

Нужно сделать как на train, так и на test датасете.
Считайте, что исходные датафреймы сохранены в `df_train` и `df_test`.
При этом в `df_train`, `df_test` должны остаться старые колонки - т.е. его нужно обогатить.


In [20]:
df_train

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
6,46,female,33.440,1,no,southeast,8240.58960
...,...,...,...,...,...,...,...
1332,52,female,44.700,3,no,southwest,11411.68500
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1336,21,female,25.800,0,no,southwest,2007.94500


In [21]:
df_test

,age,sex,bmi,children,smoker,region,charges
578,52,male,30.200,1,no,southwest,9724.53000
610,47,female,29.370,1,no,southeast,8547.69130
569,48,male,40.565,2,yes,northwest,45702.02235
1034,61,male,38.380,0,no,northwest,12950.07120
198,51,female,18.050,0,no,northwest,9644.25250
...,...,...,...,...,...,...,...
668,62,male,32.015,0,yes,northeast,45710.20785
1084,62,female,30.495,2,no,northwest,15019.76005
726,41,male,28.405,1,no,northwest,6664.68595
1132,57,male,40.280,0,no,northeast,20709.02034


In [22]:
# OHE ПЕРЕВОД В 0 1 0

df_train = pd.get_dummies(df_train, columns=['sex', 'region', 'smoker'])
# df_train = pd.concat([df_train_ohe, df_train[['sex', 'region', 'smoker']]], axis=1)

df_test = pd.get_dummies(df_test, columns=['sex', 'region', 'smoker'])
# df_test = pd.concat([df_test_ohe, df_test[['sex', 'region', 'smoker']]], axis=1)
print(df_train.columns)


Index(['age', 'bmi', 'children', 'charges', 'sex_female', 'sex_male',
       'region_northeast', 'region_northwest', 'region_southeast',
       'region_southwest', 'smoker_no', 'smoker_yes'],
      dtype='object')


In [23]:
df_test

,age,bmi,children,charges,sex_female,sex_male,region_northeast,region_northwest,region_southeast,region_southwest,smoker_no,smoker_yes
578,52,30.200,1,9724.53000,False,True,False,False,False,True,True,False
610,47,29.370,1,8547.69130,True,False,False,False,True,False,True,False
569,48,40.565,2,45702.02235,False,True,False,True,False,False,False,True
1034,61,38.380,0,12950.07120,False,True,False,True,False,False,True,False
198,51,18.050,0,9644.25250,True,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
668,62,32.015,0,45710.20785,False,True,True,False,False,False,False,True
1084,62,30.495,2,15019.76005,True,False,False,True,False,False,True,False
726,41,28.405,1,6664.68595,False,True,False,True,False,False,True,False
1132,57,40.280,0,20709.02034,False,True,True,False,False,False,True,False


### Задание №4:
Нормализуйте колонки, которые вы отметили в квизе.
Считайте, что исходные датафреймы сохранены в `df_train` и `df_test`.

Сдайте код, который модифицирует `df_train` и `df_test` так, чтобы численные колонки из прошлого пункта стали нормированы.

In [24]:
# НОРМАЛИЗАЦИЯ

# выбираем колонки для нормализации
num_cols = ["age", "bmi", "charges"]

# нормализация это процесс от колонки отнять средние значения и результат разделить на стандартную ошибку
# считает для каждой колонки из списка среднее
train_mean = df_train[num_cols].mean()
# считает стандартное отклонение, на сколько данные разбросаны вокруг среднего
train_std = df_train[num_cols].std()


# колонка - среднее / станлартную ошибку по колонке
df_train[num_cols] = (df_train[num_cols] - train_mean) / train_std
# !!! Нормализируем колонки второго датасета по значениям стд.откл и среднего из !!! трейна (предыдущего)
# Это нужно чтобы избежать утекания данных
df_test[num_cols] = (df_test[num_cols] - train_mean) / train_std



In [25]:
# !!! Целевая переменная определилась тут
# Из df_train удалился столбец "charges".
# Значения "charges" сохранились в y_train.

y_train = df_train.pop("charges")
y_test = df_test.pop("charges")

In [26]:
y_train

,charges
0,0.307931
2,-0.730599
3,0.733813
4,-0.779255
6,-0.413988
...,...
1332,-0.149159
1333,-0.216900
1334,-0.917961
1336,-0.934499


### Задание №5
Реализуйте функцию, считающую `MSE` метрику.

Ваша функция должна уметь принимать torch.Tensor, numpy-массивы и pd.Series.

In [27]:
# ФУНКЦИЯ СЧИТАЕТ MSE (Mean Squared Error) или по-русски среднеквадратичная ошибка
# это один из самых простых и популярных способов измерения качества работы модели.
def metric(preds, y):
    return ((preds - y) ** 2).mean()

# берет предсказанные значения и истинные значения

### Задание №6
Реализуйте бейзлайн на `LinearRegression` и `GradientBoostingRegressor`, отправьте метрики в ЛМС.

Используйте гиперпараметры по-умолчанию в обоих моделях.

In [28]:
# На этот момент у нас есть 2 датафрейма тестовый и тренировочный
# Они проведены через OHE
# Данные 3х столбуов нормализованы
# Есть 2 целевые метрики из тестового и трейн датафрейма, выделены в отделньый


from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

# Обучение линейной регрессии
linreg = LinearRegression() # создаем объект модели
linreg.fit(df_train, y_train) # обучение модель на датасете df_train и  целевой метриук y_train


# MSE на train
y_train_pred_linreg = linreg.predict(df_train) # делает предсказания для каждого примера из df_train (она на нем же и обучалась)
# МЫ ЕЕ ОПРЕДЕЛИИ ШАГОМ НАЗАД
# Функция metric() вычисляет среднеквадратичную ошибку (MSE) между предсказанными значениями y_train_pred_linreg и реальными значениями y_train.
mse_train_linreg = metric(y_train_pred_linreg, y_train)


################################################


# Обучение градиентного бустинга
gradboost = GradientBoostingRegressor()
gradboost.fit(df_train, y_train)

# MSE на train
y_train_pred_gradboost = gradboost.predict(df_train)
mse_train_gradboost = metric(y_train_pred_gradboost, y_train)


################################################

# Вывод только train-метрик
print(r"МСЕ линейной регрессии ", mse_train_linreg)
print(r"МСЕ градбуста ",mse_train_gradboost)
print()
# для пояснения
print(r"Это размер df_train ", df_train.shape)
print(r"Это размер предсказаний параметра y_train_pred_linreg ", y_train_pred_linreg.shape)

print("Размер предсказаний и размер датасета совпадают, так как для каждой строчки свое предсказаний. Их можно глянуть в y_train_pred_linreg")

МСЕ линейной регрессии  0.26270477886000854
МСЕ градбуста  0.10199458102652596

Это размер df_train  (1071, 11)
Это размер предсказаний параметра y_train_pred_linreg  (1071,)
Размер предсказаний и размер датасета совпадают, так как для каждой строчки свое предсказаний. Их можно глянуть в y_train_pred_linreg


In [29]:
# Запуск обученной модели на тестовых данных

y_train_pred_gradboost = gradboost.predict(df_test)
mse_train_gradboost = metric(y_train_pred_gradboost, y_test)
print(r"МСЕ градбуста ",mse_train_gradboost)

МСЕ градбуста  0.11491282212456433


In [30]:
y_train_pred_linreg = linreg.predict(df_test) # делает предсказания для каждого примера из df_train (она на нем же и обучалась)
# МЫ ЕЕ ОПРЕДЕЛИИ ШАГОМ НАЗАД
# Функция metric() вычисляет среднеквадратичную ошибку (MSE) между предсказанными значениями y_train_pred_linreg и реальными значениями y_train.
mse_train_linreg = metric(y_train_pred_linreg, y_test)
print(r"МСЕ линейной регрессии ", mse_train_linreg)

МСЕ линейной регрессии  0.2228159571271522


In [30]:
print(r"МСЕ линейной регрессии ", mse_train_linreg)
print(r"МСЕ градбуста ",mse_train_gradboost)

### Задание №7
Вычислите среднее значение целевой переменной на тренировочной выборке (train).

Подсчитайте MSE при константном предсказании этим средним и отправьте его в ЛМС.

In [31]:
########## Это БАЗОВЫЙ baseline показатель качества (MSE) для простейшего метода предсказания.
# Если MSE модели оказывается значительно ниже, чем MSE baseline, это означает, что модель действительно умеет уловить зависимости в данных.
#Если же разница невелика, возможно, модель не эффективнее простого среднего.



# Среднее значение целевой метрики
mean_train = y_train.mean()

# МСЕ
metric(mean_train, y_test)

# тут функция МСЕ которую мы написали ранее берет среднюю по всем предсказаниям и считает МСЕ по тестовой выборке

1.1145089956822465


### Задание №8
Создайте сеть, состоящую из одного слоя Linear, залейте в лмс код, который в `model` запишет вашу модель

В качестве признаков используйте все колонки в текущем датасете, за исключением таргета

In [32]:
# Да, именно так. Эта модель состоит из одного линейного слоя, который принимает вектор с 11 признаками
# и преобразует его в одно скалярное значение (1 нейрон на выходе).
# Это эквивалентно одному нейрону, который получает 11 входов.


import torch.nn as nn
import torch

# Нейросети объявляются через класс, который наследуется от torch.nn.Module
# Внутри класса нужно реализовать метод forward: в нем прописано, что сеть делает с тензором
class build_model(nn.Module):
    # В __init__ объявим все слои, которые нам нужны
    def __init__(self):
        super().__init__()

        # объявляется первый слой. Ему задается размерность. На вход 11(признаков), на выход 1(предсказание)
        # для задания нам нужно сделать только один слой
        self.linear = nn.Linear(in_features=11, out_features=1)


# посл
    def forward(self, x: torch.Tensor):
        # В этом методе нужно сделать все преобразования над входным тензором.
        return self.linear(x)
        # Да, именно так. Вызов self.linear(x) внутри метода forward выполняет все операции:
        # умножение входного тензора на матрицу весов (с транспонированием) и прибавление смещения (bias), если оно задано.
        # Это поведение описано внутри класса torch.nn.Linear и функции torch.nn.functional.linear.


# ТУТ ЗАПУСКАЕТСЯ ИНИТ и запоминает слои и прочее в функции __init__
model = build_model()
# технически ничего не происходит после такого вызова модели, только созздается объект модели и происходит расставление весов


In [33]:
import torch

torch.random.manual_seed(seed)
t_x_train = torch.from_numpy(df_train.to_numpy().astype(float)).to(dtype=torch.float32)
t_y_train = torch.from_numpy(y_train.to_numpy()).to(dtype=torch.float32)

### Задание №9
Напишите функцию `train_loop`, которая будет учить модель по данным на 2к итераций.
Считайте, что данные уже хранятся в переменных `t_x_train`, `t_y_train`.
Ваша функция `train_loop` должна вернуть список из лоссов на каждой итерации (т.е. список длины 2000).

Используйте `learning_rate=1e-2` в оптимизаторе.

Для простоты за одну итерацию делайте проход вперед и проход назад на всех наших обучающих данных.
Это будет полный градиентный спуск (не по батчам) - можем себе позволить, данных немного.

_Подсказка 1_: Вам не обязательно учить модель на видеокарте, CPU будет достаточно.

_Подсказка 2_: `tqdm` - это библиотека, которая рисует прогресс итераций

In [34]:
# штука чтобы показывать прогресс в цикле
import tqdm

# все методы оптимизации лежат в torch.optim
# импортируется стохастический градиентный спуск, который будет обновлять параметры модели
from torch.optim.sgd import SGD
#  содержит функциональные интерфейсы для вычисления функций потерь и других операций.
import torch.nn.functional as F


# создаем optimizer и говорим "ты отвечаешь за все параметры модели"
optimizer = SGD(params=model.parameters(), lr=1e-2)
# model.parameters() возвращает все обучаемые параметры модели (в виде тензоров).
# Откуда функция все эти параметры узнает, ведь мы нигде их явно не прописывали в классе модели?
# Это магия PyTorch, он умеет многое делать сам :)


def train_loop(model: nn.Module, optimizer: SGD, x: torch.Tensor, y: torch.Tensor) -> list[float]:
    losses = []
    loss_fn = nn.MSELoss()
    # Пройдемся 2000 раз по всем данным
    for _ in tqdm.trange(2000):
        # optimizer по умолчанию "помнит" градиенты с прошлых итераций - и прибавляет к новым.
        # Так сделано ради продвинутых техник обучения.
        # Нам это не надо, поэтому в каждой итерации явно зануляем все градиенты - пусть считаются заново.
        optimizer.zero_grad()

        # Это блок уже знаем: считаем выход, потери, градиенты по потерям.
        # Forward pass: вычисляем предсказания модели
        output = model(x)

        # Вычисляем лосс
        loss = loss_fn(output.squeeze(), y) # Убираем лишние размерности, если они есть

        # Backward pass: вычисляем градиенты
        loss.backward()

        # Просим оптимизатор пройтись по параметрам и сделать градиентный спуск.
        # Оптимизатор сам обновит веса, вручную этого делать не надо
        optimizer.step()

        # Запомним loss
        losses.append(loss.detach().item())
    return losses


losses = train_loop(model, optimizer, t_x_train, t_y_train)

100%|██████████| 2000/2000 [00:01<00:00, 1289.42it/s]


In [35]:
# import tqdm
# from torch.optim.sgd import SGD


# def train_loop(model: nn.Module) -> list[float]:
#     losses = []
#     ...
#     return losses

# это было как ориентир

### Задание №10
Обучите модель, состоящую из одного слоя `Linear`.
Приложите в ЛМС метрику `MSE` на тестовых данных.
Используйте `learning_rate=1e-2` в оптимизаторе.

Когда будете тестировать, не забудьте перенести тестовые данные в `torch.Tensor`

In [37]:
torch.random.manual_seed(seed)
t_x_test = torch.from_numpy(df_test.to_numpy().astype(float)).to(dtype=torch.float32)
t_y_test = torch.from_numpy(y_test.to_numpy()).to(dtype=torch.float32)

In [38]:
model.eval()
with torch.no_grad():
  y_pred_nn = model(t_x_test)
mse_nn = metric(y_pred_nn[:,0], t_y_test)
mse_nn.item()

0.22281694412231445

### Задание №11
Вам необходимо усложнить существующую нейронную сеть, добавив один скрытый слой.

Используйте следующие параметры:

Размерность скрытого слоя: 6 нейронов, функция активации -  `ReLU`

Приложите в лмс код, который в переменную `model` запишет вашу модель

In [39]:
def build_model():
  model = nn.Sequential(
      nn.Linear(11, 6),
      nn.ReLU(),
      nn.Linear(6,1)
  )
  return model

model = build_model()

### Задание №12
Приложите в ЛМС метрику качества этой сети после 2к итераций обучения.
Эту модель можно обучить на CPU, не обязательно на видеокарте.

Используйте для обучения ту же функцию `train_loop` с теми же параметрами (`learning rate`, число итераций и т.п.)

In [40]:
# штука чтобы показывать прогресс в цикле
import tqdm

# все методы оптимизации лежат в torch.optim
# импортируется стохастический градиентный спуск, который будет обновлять параметры модели
from torch.optim.sgd import SGD
#  содержит функциональные интерфейсы для вычисления функций потерь и других операций.
import torch.nn.functional as F


# создаем optimizer и говорим "ты отвечаешь за все параметры модели"
optimizer = SGD(params=model.parameters(), lr=1e-2)
# model.parameters() возвращает все обучаемые параметры модели (в виде тензоров).
# Откуда функция все эти параметры узнает, ведь мы нигде их явно не прописывали в классе модели?
# Это магия PyTorch, он умеет многое делать сам :)


def train_loop(model: nn.Module, optimizer: SGD, x: torch.Tensor, y: torch.Tensor) -> list[float]:
    losses = []
    loss_fn = nn.MSELoss()
    # Пройдемся 2000 раз по всем данным
    for _ in tqdm.trange(2000):
        # optimizer по умолчанию "помнит" градиенты с прошлых итераций - и прибавляет к новым.
        # Так сделано ради продвинутых техник обучения.
        # Нам это не надо, поэтому в каждой итерации явно зануляем все градиенты - пусть считаются заново.
        optimizer.zero_grad()

        # Это блок уже знаем: считаем выход, потери, градиенты по потерям.
        # Forward pass: вычисляем предсказания модели
        output = model(x)

        # Вычисляем лосс
        loss = loss_fn(output.squeeze(), y) # Убираем лишние размерности, если они есть

        # Backward pass: вычисляем градиенты
        loss.backward()

        # Просим оптимизатор пройтись по параметрам и сделать градиентный спуск.
        # Оптимизатор сам обновит веса, вручную этого делать не надо
        optimizer.step()

        # Запомним loss
        losses.append(loss.detach().item())
    return losses



losses = train_loop(model, optimizer, t_x_train, t_y_train)

100%|██████████| 2000/2000 [00:06<00:00, 302.44it/s]


In [41]:
model.eval()
with torch.no_grad():
  y_pred_nn = model(t_x_test)
mse_nn = metric(y_pred_nn[:,0], t_y_test)
mse_nn.item()

0.1472102850675583

### Задание №13
Добавьте дополнительные слои в нейронную сеть

Вам необходимо усложнить нейронную сеть, добавив еще 2-3 скрытых слоя с такими же размерностями, как в предыдущем задании.

Приложите в лмс код, который в переменную `model` запишет вашу модель

In [43]:
def build_model():
  model = nn.Sequential(
      nn.Linear(11, 6),
      nn.ReLU(),
      nn.Linear(6,6),
      nn.Linear(6,6),
      nn.Linear(6,1)
  )
  return model

model = build_model()

### Задание №14
Приложите в ЛМС метрику качества после 2к итераций обучения.

Используйте для обучения ту же функцию `train_loop` с теми же параметрами (`learning rate`, число итераций и т.п.)

Эту модель можно обучить на CPU, не обязательно на видеокарте.

In [44]:
# штука чтобы показывать прогресс в цикле
import tqdm

# все методы оптимизации лежат в torch.optim
# импортируется стохастический градиентный спуск, который будет обновлять параметры модели
from torch.optim.sgd import SGD
#  содержит функциональные интерфейсы для вычисления функций потерь и других операций.
import torch.nn.functional as F


# создаем optimizer и говорим "ты отвечаешь за все параметры модели"
optimizer = SGD(params=model.parameters(), lr=1e-2)
# model.parameters() возвращает все обучаемые параметры модели (в виде тензоров).
# Откуда функция все эти параметры узнает, ведь мы нигде их явно не прописывали в классе модели?
# Это магия PyTorch, он умеет многое делать сам :)


def train_loop(model: nn.Module, optimizer: SGD, x: torch.Tensor, y: torch.Tensor) -> list[float]:
    losses = []
    loss_fn = nn.MSELoss()
    # Пройдемся 2000 раз по всем данным
    for _ in tqdm.trange(2000):
        # optimizer по умолчанию "помнит" градиенты с прошлых итераций - и прибавляет к новым.
        # Так сделано ради продвинутых техник обучения.
        # Нам это не надо, поэтому в каждой итерации явно зануляем все градиенты - пусть считаются заново.
        optimizer.zero_grad()

        # Это блок уже знаем: считаем выход, потери, градиенты по потерям.
        # Forward pass: вычисляем предсказания модели
        output = model(x)

        # Вычисляем лосс
        loss = loss_fn(output.squeeze(), y) # Убираем лишние размерности, если они есть

        # Backward pass: вычисляем градиенты
        loss.backward()

        # Просим оптимизатор пройтись по параметрам и сделать градиентный спуск.
        # Оптимизатор сам обновит веса, вручную этого делать не надо
        optimizer.step()

        # Запомним loss
        losses.append(loss.detach().item())
    return losses



losses = train_loop(model, optimizer, t_x_train, t_y_train)

100%|██████████| 2000/2000 [00:04<00:00, 456.79it/s]


In [45]:
model.eval()
with torch.no_grad():
  y_pred_nn = model(t_x_test)
mse_nn = metric(y_pred_nn[:,0], t_y_test)
mse_nn.item()

0.13172727823257446

In [48]:
print(r"МСЕ градбуста ",mse_train_gradboost)
print(r"нейросетевое", mse_nn.item())
print(r"МСЕ линейной регрессии ", mse_train_linreg)


МСЕ градбуста  0.11491282212456433
нейросетевое 0.13172727823257446
МСЕ линейной регрессии  0.2228159571271522
